# Script to compute Panoptic Quality (PQ) metric for each image of each participant and to store results in an excel file


## Created by Ruchika Verma

This code will generate an excel file containing multiple sheets (named as participants team-names). Each sheet will save image names in rows and respective PQ metrics in columns.

`Epithelial` = column 1, `Lymphocyte`  = column 2, `Macrophage` = column 3, `Neutrophil` = column 4

Note: This code will work if n-ary masks are stored in both ground truth and predicted path. If the mask is stored as binary, it will first convert it into n-ary and then compute PQ metric
 
Please run n-ary mask generation code from [here](https://github.com/ruchikaverma-iitg/MoNuSAC/blob/master/n-ary_mask_generation.ipynb) to see ground truth and predicted masks format.

The [code](https://github.com/ruchikaverma-iitg/MoNuSAC/blob/master/n-ary_mask_generation.ipynb) will save masks to compute PQ metric as given below:<br>
-`Folder` -> Patient name <br>
-`Sub-folder` -> Sub-images under each patient<br>
-`Sub-Sub-folder` -> Annotated cell-type on each sub-image which contains n-ary masks (saved as mat file)

### Input
- ground_truth_path: Path to read ground truth masks from <br>
- Predicted_path: Path to read participant folders from <br>

### Output
An excel file with name `MoNuSAC-testing-PQ.xls` will store on the given ground_truth_path

### Reference
Kirillov, A., He, K., Girshick, R., Rother, C., & Dollár, P. (2019). Panoptic segmentation. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 9404-9413).

In [24]:
import os
import numpy as np
import glob
import cv2
import scipy.io as sio
from PIL import Image
import scipy
import scipy.ndimage
import xlwt 
from xlwt import Workbook 

In [25]:
# Compute Panoptic quality metric for each image
def Panoptic_quality(ground_truth_image,predicted_image):
    TP = 0
    FP = 0
    FN = 0
    sum_IOU = 0
    matched_instances = {}# Create a dictionary to save ground truth indices in keys and predicted matched instances as velues
                        # It will also save IOU of the matched instance in [indx][1]

    # Find matched instances and save it in a dictionary
    for i in np.unique(ground_truth_image):
        if i == 0:
            pass
        else:
            temp_image = np.array(ground_truth_image)
            temp_image = temp_image == i
            matched_image = temp_image * predicted_image
        
            for j in np.unique(matched_image):
                if j == 0:
                    pass
                else:
                    pred_temp = predicted_image == j
                    intersection = sum(sum(temp_image*pred_temp))
                    union = sum(sum(temp_image + pred_temp))
                    IOU = intersection/union
                    if IOU> 0.5:
                        matched_instances [i] = j, IOU 
                        
    # Compute TP, FP, FN and sum of IOU of the matched instances to compute Panoptic Quality               
                        
    pred_indx_list = np.unique(predicted_image)
    pred_indx_list = np.array(pred_indx_list[1:])

    # Loop on ground truth instances
    for indx in np.unique(ground_truth_image):
        if indx == 0:
            pass
        else:
            if indx in matched_instances.keys():
                pred_indx_list = np.delete(pred_indx_list, np.argwhere(pred_indx_list == [indx][0]))
                TP = TP+1
                sum_IOU = sum_IOU+matched_instances[indx][1]
            else:
                FN = FN+1
    FP = len(np.unique(pred_indx_list))
    PQ = sum_IOU/(TP+0.5*FP+0.5*FN)
    
    return PQ

In [26]:
ground_truth_path = 'D:\Personal improvisation\IITB\MoNuSAC\Submissions\Mask_files\Organizers_MoNuSAC_test_results' #Ground truth path to read data from
Predicted_path = 'D:\Personal improvisation\IITB\MoNuSAC\Submissions\Mask_files\Predicted_updated_masks' #Path to read predicted outcomes from

import os
os.chdir(ground_truth_path)

In [27]:
participants_folders=glob.glob(Predicted_path+'/**')
participants_folders

['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod',
 'D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Dark Brotherhood',
 'D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\IVG',
 'D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426']

In [28]:
cell_types = ['Epithelial','Lymphocyte', 'Macrophage', 'Neutrophil']

In [29]:
files=glob.glob('./**/**')
len(files) #Ground Truth files

101

In [30]:
# Workbook is created
wb = Workbook() 
    
for participant_folder in participants_folders:
    print(participant_folder[86:]) # To save only team name as an excel sheet instead of path of the folder containing predicted masks 
    
    # add_sheet is used to create sheet of each participant
    ccbt = wb.add_sheet(participant_folder[86:]) 
    ccbt.write(0, 0, 'Patient ID')
    ccbt.write(0, 1, 'Epithelial') 
    ccbt.write(0, 2, 'Lymphocyte') 
    ccbt.write(0, 3, 'Neutrophil') 
    ccbt.write(0, 4, 'Macrophage') 
    
    for image_count, filei in enumerate(files):
        ccbt.write(image_count+1,0, filei) # Add image name in excel file
    
        ## Ambiguous_region which was provided with the testing data to exclude from the metric computation
        imgs=glob.glob(filei+'/**/**')
        ambiguous_idx = [i for i, f_name in enumerate(imgs) if 'Ambiguous' in f_name]
    
        # Check if abmiguous_idx exists
        if ambiguous_idx:
            ambiguous_regions = sio.loadmat(imgs[ambiguous_idx[0]])['n_ary_mask']
            ambiguous_regions = 1-(ambiguous_regions > 0)
            imgs.pop(ambiguous_idx[0])    
         
        for i, f_name in enumerate(imgs):
            print(f_name)
            class_id = ([idx for idx in range(len(cell_types)) if cell_types[idx] in f_name])# Cell-type
             
            # Read ground truth image    
            ground_truth = sio.loadmat(f_name)['n_ary_mask']
            
            # Read predicted mask and exclude the ambiguous regions for metric computation   
            pred_img_name = glob.glob(participant_folder+'/'+filei+'/'+cell_types[class_id[0]]+'/**')  
            
            if not pred_img_name:
                ccbt.write(image_count+1, class_id[0]+1, 0)
                print(0)
            else:
                predicted_mask = sio.loadmat(pred_img_name[0])
              
                mask_saved = ['img','name','n_ary_mask','Neutrophil_mask','arr','item','Epithelial_mask','Macrophage_mask', 'Lymphocyte_mask']
                mask_key = [m for m in mask_saved if m in predicted_mask.keys()]
                predicted_mask = predicted_mask[mask_key[0]]
                
                # Converting binary to n-ary mask for those participants who did not send masks as informed
                if (len(np.unique(predicted_mask)) == 2):
                    predicted_mask, num_features = scipy.ndimage.measurements.label(predicted_mask)
                    
            
                print(pred_img_name)
        
                if ambiguous_idx:
                    predicted_mask = predicted_mask*ambiguous_regions
            
                # Compute Panoptic Quality
                PQ = Panoptic_quality(ground_truth,predicted_mask)
                print(PQ)
        
                ccbt.write(image_count+1, class_id[0]+1, PQ)    
    
wb.save('MoNuSAC-testing-PQ.xls')    

Amirreza Mahbod
.\TCGA-2Z-A9JG-01Z-00-DX1\TCGA-2Z-A9JG-01Z-00-DX1_1\Neutrophil\3_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-2Z-A9JG-01Z-00-DX1\\TCGA-2Z-A9JG-01Z-00-DX1_1/Neutrophil\\TCGA-2Z-A9JG-01Z-00-DX1_1.mat']
0.168600154679041
.\TCGA-2Z-A9JG-01Z-00-DX1\TCGA-2Z-A9JG-01Z-00-DX1_2\Neutrophil\8_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-2Z-A9JG-01Z-00-DX1\\TCGA-2Z-A9JG-01Z-00-DX1_2/Neutrophil\\TCGA-2Z-A9JG-01Z-00-DX1_2.mat']
0.5242812742812742
.\TCGA-2Z-A9JG-01Z-00-DX1\TCGA-2Z-A9JG-01Z-00-DX1_3\Neutrophil\13_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-2Z-A9JG-01Z-00-DX1\\TCGA-2Z-A9JG-01Z-00-DX1_3/Neutrophil\\TCGA-2Z-A9JG-01Z-00-DX1_3.mat']
0.0
.\TCGA-2Z-A9JG-01Z-00-DX1\TCGA-2Z-A9JG-01Z-00-DX1_4\Epithelial\16_mask.mat
['D:\\Personal imp

0.4595111391948614
.\TCGA-55-7570-01Z-00-DX1\TCGA-55-7570-01Z-00-DX1_002\Lymphocyte\87_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-55-7570-01Z-00-DX1\\TCGA-55-7570-01Z-00-DX1_002/Lymphocyte\\TCGA-55-7570-01Z-00-DX1_002.mat']
0.0
.\TCGA-55-7570-01Z-00-DX1\TCGA-55-7570-01Z-00-DX1_002\Macrophage\89_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-55-7570-01Z-00-DX1\\TCGA-55-7570-01Z-00-DX1_002/Macrophage\\TCGA-55-7570-01Z-00-DX1_002.mat']
0.5033268631067883
.\TCGA-55-7573-01Z-00-DX1\TCGA-55-7573-01Z-00-DX1_001\Lymphocyte\92_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-55-7573-01Z-00-DX1\\TCGA-55-7573-01Z-00-DX1_001/Lymphocyte\\TCGA-55-7573-01Z-00-DX1_001.mat']
0.8187134502923976
.\TCGA-55-7573-01Z-00-DX1\TCGA-55-7573-01Z-00-DX1_001\Macrophage\94_m

0.47603153728323405
.\TCGA-D8-A3Z6-01Z-00-DX2\TCGA-D8-A3Z6-01Z-00-DX2_002\Epithelial\181_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-D8-A3Z6-01Z-00-DX2\\TCGA-D8-A3Z6-01Z-00-DX2_002/Epithelial\\TCGA-D8-A3Z6-01Z-00-DX2_002.mat']
0.5264070125293737
.\TCGA-D8-A3Z6-01Z-00-DX2\TCGA-D8-A3Z6-01Z-00-DX2_002\Lymphocyte\182_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-D8-A3Z6-01Z-00-DX2\\TCGA-D8-A3Z6-01Z-00-DX2_002/Lymphocyte\\TCGA-D8-A3Z6-01Z-00-DX2_002.mat']
0.3450007274572054
.\TCGA-D8-A3Z6-01Z-00-DX2\TCGA-D8-A3Z6-01Z-00-DX2_003\Lymphocyte\187_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-D8-A3Z6-01Z-00-DX2\\TCGA-D8-A3Z6-01Z-00-DX2_003/Lymphocyte\\TCGA-D8-A3Z6-01Z-00-DX2_003.mat']
0.48031896035974603
.\TCGA-D8-A3Z6-01Z-00-DX2\TCGA-D8-A3Z6-01Z-00-DX2

['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-E2-A108-01Z-00-DX1\\TCGA-E2-A108-01Z-00-DX1_005/Lymphocyte\\TCGA-E2-A108-01Z-00-DX1_005.mat']
0.4249304125146134
.\TCGA-EW-A6SB-01Z-00-DX1\TCGA-EW-A6SB-01Z-00-DX1_001\Epithelial\271_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-EW-A6SB-01Z-00-DX1\\TCGA-EW-A6SB-01Z-00-DX1_001/Epithelial\\TCGA-EW-A6SB-01Z-00-DX1_001.mat']
0.5262077170199413
.\TCGA-EW-A6SB-01Z-00-DX1\TCGA-EW-A6SB-01Z-00-DX1_001\Lymphocyte\272_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-EW-A6SB-01Z-00-DX1\\TCGA-EW-A6SB-01Z-00-DX1_001/Lymphocyte\\TCGA-EW-A6SB-01Z-00-DX1_001.mat']
0.37016785891818793
.\TCGA-EW-A6SB-01Z-00-DX1\TCGA-EW-A6SB-01Z-00-DX1_002\Epithelial\276_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\

.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_5\Neutrophil\363_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-G9-6367-01Z-00-DX1\\TCGA-G9-6367-01Z-00-DX1_5/Neutrophil\\TCGA-G9-6367-01Z-00-DX1_5.mat']
0.5303439456820693
.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_6\Lymphocyte\367_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-G9-6367-01Z-00-DX1\\TCGA-G9-6367-01Z-00-DX1_6/Lymphocyte\\TCGA-G9-6367-01Z-00-DX1_6.mat']
0.0
.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_6\Neutrophil\368_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\Amirreza Mahbod/.\\TCGA-G9-6367-01Z-00-DX1\\TCGA-G9-6367-01Z-00-DX1_6/Neutrophil\\TCGA-G9-6367-01Z-00-DX1_6.mat']
0.5949976272489251
.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_7\Epithelial\371_mask.mat
['D:\\Personal improvisatio

...

0.5655692657174526
.\TCGA-2Z-A9JG-01Z-00-DX1\TCGA-2Z-A9JG-01Z-00-DX1_6\Neutrophil\28_mask.mat
0
.\TCGA-2Z-A9JN-01Z-00-DX1\TCGA-2Z-A9JN-01Z-00-DX1_1\Lymphocyte\32_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-2Z-A9JN-01Z-00-DX1\\TCGA-2Z-A9JN-01Z-00-DX1_1/Lymphocyte\\14.mat']
0.6716333678966733
.\TCGA-2Z-A9JN-01Z-00-DX1\TCGA-2Z-A9JN-01Z-00-DX1_2\Epithelial\36_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-2Z-A9JN-01Z-00-DX1\\TCGA-2Z-A9JN-01Z-00-DX1_2/Epithelial\\15.mat']
0.5462752183189036
.\TCGA-2Z-A9JN-01Z-00-DX1\TCGA-2Z-A9JN-01Z-00-DX1_3\Neutrophil\43_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-2Z-A9JN-01Z-00-DX1\\TCGA-2Z-A9JN-01Z-00-DX1_3/Neutrophil\\17.mat']
0.0
.\TCGA-2Z-A9JN-01Z-00-DX1\TCGA-2Z-A9JN-01Z-00-DX1_4\Macrophage\49_mask.mat
0
.\TCGA-49-6743-01Z-00-D

0.37603577039008307
.\TCGA-A2-A04X-01Z-00-DX1\TCGA-A2-A04X-01Z-00-DX1_001\Lymphocyte\137_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-A2-A04X-01Z-00-DX1\\TCGA-A2-A04X-01Z-00-DX1_001/Lymphocyte\\61.mat']
0.69076835304952
.\TCGA-A2-A04X-01Z-00-DX1\TCGA-A2-A04X-01Z-00-DX1_002\Epithelial\141_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-A2-A04X-01Z-00-DX1\\TCGA-A2-A04X-01Z-00-DX1_002/Epithelial\\62.mat']
0.5296484064077933
.\TCGA-A2-A04X-01Z-00-DX1\TCGA-A2-A04X-01Z-00-DX1_002\Macrophage\144_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-A2-A04X-01Z-00-DX1\\TCGA-A2-A04X-01Z-00-DX1_002/Macrophage\\64.mat']
0.1549764826684649
.\TCGA-A2-A04X-01Z-00-DX1\TCGA-A2-A04X-01Z-00-DX1_003\Neutrophil\148_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\

0.5106173369925281
.\TCGA-E2-A108-01Z-00-DX1\TCGA-E2-A108-01Z-00-DX1_001\Lymphocyte\247_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-E2-A108-01Z-00-DX1\\TCGA-E2-A108-01Z-00-DX1_001/Lymphocyte\\108.mat']
0.7014510713664255
.\TCGA-E2-A108-01Z-00-DX1\TCGA-E2-A108-01Z-00-DX1_002\Epithelial\251_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-E2-A108-01Z-00-DX1\\TCGA-E2-A108-01Z-00-DX1_002/Epithelial\\109.mat']
0.803526092268084
.\TCGA-E2-A108-01Z-00-DX1\TCGA-E2-A108-01Z-00-DX1_002\Lymphocyte\252_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-E2-A108-01Z-00-DX1\\TCGA-E2-A108-01Z-00-DX1_002/Lymphocyte\\110.mat']
0.4066265060240964
.\TCGA-E2-A108-01Z-00-DX1\TCGA-E2-A108-01Z-00-DX1_003\Epithelial\256_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_file

0.4104458103939163
.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_1\Lymphocyte\342_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-G9-6367-01Z-00-DX1\\TCGA-G9-6367-01Z-00-DX1_1/Lymphocyte\\149.mat']
0.7754251933283804
.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_1\Neutrophil\343_mask.mat
0
.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_2\Epithelial\346_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-G9-6367-01Z-00-DX1\\TCGA-G9-6367-01Z-00-DX1_2/Epithelial\\150.mat']
0.7492557325014552
.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_2\Lymphocyte\347_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-G9-6367-01Z-00-DX1\\TCGA-G9-6367-01Z-00-DX1_2/Lymphocyte\\151.mat']
0.2737226277372263
.\TCGA-G9-6367-01Z-00-DX1\TCGA-G9-6367-01Z-00-DX1_3\Macrophage\354_mask.mat
['

0.7680020814149008
.\TCGA-VP-A87E-01Z-00-DX1\TCGA-VP-A87E-01Z-00-DX1_2\Lymphocyte\457_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-VP-A87E-01Z-00-DX1\\TCGA-VP-A87E-01Z-00-DX1_2/Lymphocyte\\189.mat']
0.8342898530846269
.\TCGA-VP-A87E-01Z-00-DX1\TCGA-VP-A87E-01Z-00-DX1_3\Neutrophil\463_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-VP-A87E-01Z-00-DX1\\TCGA-VP-A87E-01Z-00-DX1_3/Neutrophil\\194.mat']
0.8081404112149857
.\TCGA-VP-A87E-01Z-00-DX1\TCGA-VP-A87E-01Z-00-DX1_4\Epithelial\466_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_updated_masks\\SJTU_426/.\\TCGA-VP-A87E-01Z-00-DX1\\TCGA-VP-A87E-01Z-00-DX1_4/Epithelial\\193.mat']
0.6802400305269343
.\TCGA-VP-A87E-01Z-00-DX1\TCGA-VP-A87E-01Z-00-DX1_4\Lymphocyte\467_mask.mat
['D:\\Personal improvisation\\IITB\\MoNuSAC\\Submissions\\Mask_files\\Predicted_